In [1]:
%cd ../..
%load_ext autoreload
%autoreload 2

REMOTE_RTS_DATA = "/media/sinergia/RTS/"
REMOTE_VIDEOS = '/mnt/rts/'

LOCAL_RTS_DATA = "/media/data/rts/"
METADATA = LOCAL_RTS_DATA + 'metadata'
LOCAL_VIDEOS = LOCAL_RTS_DATA + 'archive'

OUTDIR = 'data'


/home/kirell/work/rts


In [2]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path

In [3]:
# LOCAL imports
import rts
import rts.pipelines.rts
import rts.utils
import rts.io.media
import rts.features.audio
import rts.features.text
import rts.features.image

LOG = rts.utils.get_logger()

In [4]:
import rts.db.queries

In [9]:
rts_lib_id = rts.db.queries.get_library_id_from_name('rts')

In [10]:
df = rts.pipelines.rts.build_metadata_df(REMOTE_VIDEOS, 
                                         LOCAL_RTS_DATA, force=False)
# sample_df = rts.pipelines.rts.get_one_percent_sample(df)
# # # rts.metadata.export_metadata_stats(sample_df, Path(OUTDIR) / 'sample')
# # # rts.metadata.export_metadata_stats(df, OUTDIR)
# sample_df.head()

# medias = rts.metadata.load_all_media_info(LOCAL_VIDEOS)
# clips = rts.metadata.load_all_clips(LOCAL_VIDEOS)

# clip_df = rts.metadata.build_clips_df(LOCAL_VIDEOS, METADATA)
# rts.utils.dataframe_to_hdf5(METADATA, 'clips', clip_df)

In [5]:
# sujets = rts.metadata.filter_by_asset_type(df, nested_struct='0/0')

In [9]:
df

,guid,mediaFolderPath,mediaDuration,ratio,formatResolution,publishedDate,categoryName,assetType,contentType,backgoundType,collection,publishedBy,rights,title,resume,geoTheme,resumeSequence,created,published,sequences
mediaId,,,,,,,,,,,,,,,,,,,,
ZE003073,AA1101000650,/mnt/rts/3/7/0/ZE003073,2932,16:9,SD,2010-10-10T20:05:00Z,Programme,Programme,Autres (information),"[BETISIER, BIO]",Mise au point,TSR 1,Restriction/Condition,PETITS MEURTRES ENTRE AMIS // UN LIT POUR DEUX...,"* SUISSE : 20101010, l'inspecteur HEIMO enquêt...",[SUISSE],[00:02.36\r\nPlateau : diverses présentations ...,1670505072,1286741100,"{'892779': {'guid': 'AA1101000650', 'mediaId':..."
ZE003216,AA1101000660,/mnt/rts/6/1/2/ZE003216,3043,16:9,SD,2010-11-14T00:00:00Z,Programme,Programme,Autres (information),None,Mise au point,TSR 1,Restriction/Condition,BERLUSCONNERIES ? // SERPENT DU LAC // L'ADIEU...,"* ITALIE : 20101114 et archives, un nouveau sc...","[VAUD, ITALIE, PLATEAU, SUISSE, SAXON, TOUR-DE...",[00:02.38\r\nPlateau : diverses présentation d...,1670505072,1289692800,"{'893397': {'guid': 'AA1101000660', 'mediaId':..."
ZE010504,AA1101000710,/mnt/rts/4/0/5/ZE010504,278,16:9,SD,2010-10-31T00:00:00Z,Programme,Programme,Autres (information),None,Ensemble,TSR 1,Inconnu,PRO JUVENTUTE - 10.10.31,"* SUISSE : 20101031, présentation de la Fondat...",[SUISSE],"[00:01.00\r\nSUISSE, int salle de classe : div...",1670505072,1288483200,"{'894120': {'guid': 'AA1101000710', 'mediaId':..."
ZE003393,AA1101000730,/mnt/rts/3/9/3/ZE003393,1922,16:9,SD,2011-01-10T00:00:00Z,Programme,Programme,Folklore et musique populaire,None,La boîte à musique,TSR 2,Inconnu,4E TROPHEE DES MUSIQUES POPULAIRES : 1ERE DEMI...,"* Moudon, Plateau : 20101218 et archives, émis...","[MOUDON, PLATEAU]","[ Moudon, diverses séquences : diverses présen...",1670505072,1294617600,"{'895543': {'guid': 'AA1101000730', 'mediaId':..."
ZE003206,AA1101000740,/mnt/rts/6/0/2/ZE003206,744,16:9,SD,2010-11-23T00:00:00Z,Programme,Programme,"Jeux, concours",None,Tapis rouge,TSR 1,Restriction/Condition,10.11.23,None,None,None,1670505072,1290470400,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZB067527,GE9912232009,/mnt/rts/7/2/5/ZB067527,595,4:3,SD,1999-12-12T00:00:00Z,Programme,Programme,Autres (information),None,Mise au point,TSR 1,Restriction/Condition,CIRCULEZ Y A RIEN A LOUER // LE PINGOUIN AUX O...,"* Genève-ville, Genève : 19991212, la ville de...","[GENEVE, LIEU INDÉTERMINÉ, TOKYO, BERNE-VILLE,...","[ Plateau : intro CHYBA, Martina, CIRCULEZ Y A...",1670505580,944956800,"{'783005': {'guid': 'GE9912232009', 'mediaId':..."
ZB025142,GE9912232399,/mnt/rts/2/4/1/ZB025142,4153,4:3,SD,1999-12-31T00:00:00Z,Programme,Programme,Shows et variétés musicales,None,Grille des fêtes,TSR 1,Inconnu,FETE DU NOUVEL AN 2000 : LE SPECTACLE DU TEMPS...,* Genève-ville : Plainpalais,[GENEVE-VILLE],"[ Genève-ville, Plainpalais, de nuit : divers ...",1670505580,946598400,"{'784358': {'guid': 'GE9912232399', 'mediaId':..."
ZB028574,GE9912237119,/mnt/rts/4/7/5/ZB028574,1429,4:3,SD,2000-02-26T00:00:00Z,Programme,Programme,Série et feuilletons,None,Les Pique-Meurons,TSR 1,Restriction/Condition,Y A COMME UN OS - 00.02.26,"* Genève-ville : 20000226, sitcom réalisé par ...",[GENEVE-VILLE],"[ Genève-ville : sitcom avec MORENA, Lolita, c...",1670505580,951523200,"{'783084': {'guid': 'GE9912237119', 'mediaId':..."


In [32]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')

outpath = Path(OUTDIR) / 'test_trim.mp4'
# ok, outpath = rts.io.media.trim(video_path,  Path(OUTDIR) / 'test_trim.mp4', 300, 310)
# print(ok, outpath)
# outpath = rts.io.media.trim(video_path,  Path(OUTDIR) / 'test_trim.mp4', 300, 310)

ok, outpath = rts.io.media.trim(video_path,  ":", 300, 310)
print(ok, outpath)
if isinstance(outpath, io.BytesIO):
    print(len(outpath.getvalue()))

True <_io.BytesIO object at 0x7fdb889a61b0>
515797


In [7]:
rts.features.image.embed_images([Path(OUTDIR) / 'testori1.jpg', 
                                 Path(OUTDIR) / 'testori2.jpg'])

INFO:RTS:Load model: resnet50


array([[0.06276648, 1.1780145 , 0.3374293 , ..., 0.2396185 , 0.16062912,
        0.41618043],
       [0.28102228, 0.5253164 , 0.09940885, ..., 0.00276155, 0.019754  ,
        0.05477377]], dtype=float32)

In [18]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')

outpath = Path(OUTDIR) / 'test_trim.mp4'
outpath = rts.io.media.trim(video_path,  Path(OUTDIR) / 'test_trim.mp4', 300, 310)


Args: ['-hide_banner', '-loglevel', 'error', '-i', '/media/data/rts/archive/0/0/0/ZB001000/ZB001000.mp4', '-filter_complex', '[0:v]trim=end=310:start=300[s0];[s0]setpts=PTS-STARTPTS[s1];[0:a]atrim=end=310:start=300[s2];[s2]asetpts=PTS-STARTPTS[s3];[s1][s3]concat=a=1:n=1:v=1[s4][s5]', '-map', '[s4]', '-map', '[s5]', '-movflags', 'faststart', 'data/test_trim.mp4', '-y']


In [ ]:
atlases = rts.metadata.create_clip_texture_atlases(df, LOCAL_RTS_DATA,
                                                   '0', # folder name
                                                   tile_size=64,
                                                   flip=True,
                                                   no_border=True, 
                                                   format='jpg')

In [ ]:
import PIL
import os
from PIL import ImageOps, Image

In [ ]:
test_im_path = os.path.join(OUTDIR, 'test_image.jpg')
im = Image.open(test_im_path)
# rts.io.media.save_image_pyramid(im, Path(OUTDIR) / 'test_image_pyramid', 'test_image', True)

In [ ]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')
# audio_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000_audio.aac')
# rts.io.media.get_media_info(video_path)

# scenes = rts.io.media.detect_scenes(video_path)
# scenes = rts.io.media.filter_scenes(scenes, 5)
# clips = rts.io.media.save_scenes_images(scenes, video_path, OUTDIR)
# rts.pipeline.save_scenes(clips, OUTDIR)

In [5]:
test_video_folder = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000')
audio_path = os.path.join(test_video_folder, 'ZB001000_audio.m4a')

# outpath = rts.io.media.trim_media(video_path, 30, 40, Path(OUTDIR) / 'test_trim.mp4')
# rts.io.media.get_media_info(outpath)

In [ ]:
clips = rts.pipeline.extract_location_clips_from_transcript(transcript,
    25,
    OUTDIR,
    video_path,
    min_seconds=3,
    extend_duration=6,
    num_images=3,
    force=True
)

In [ ]:
imgs = list(rts.utils.recursive_glob(Path(OUTDIR) / 'images' / 'original', '.jpg'))
imgs

In [ ]:
rts.io.media.create_square_atlases(
    imgs,
    Path(OUTDIR) / 'atlas',
    width=512,
    max_tile_size=128,
    no_border=True,
    flip=True,
    atlas_prefix='small_atlas',
    bg_color=(255, 255, 255, 255)
)

In [ ]:
res = rts.features.text.enrich_scenes_with_transcript(clips.copy(), t)
# res

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
input_video_folder = os.path.join(REMOTE_VIDEOS, '0', '0', '0', 'ZB001000')
# input_video_folder = os.path.join(VIDEOS, '0', '0', 'ZB002000')
# v, a = rts.io.archive.get_raw_video_audio_parts(input_video_folder)

# '/'.join(input_video_folder.split('/')[-4:])
# remuxed = rts.pipeline.create_optimized_media(input_video_folder, LOCAL_VIDEOS, force=True)
# print(remuxed)

rts.pipeline.process_media(input_video_folder, LOCAL_VIDEOS, 
    compute_transcript=True, force_scene=False, force_trans=False)

In [ ]:
%ls {remux_folder}

In [ ]:
fts.groupby('mediaId').count().sort_values('location', ascending=False)

In [ ]:
fts

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
rts.utils.obj_to_json(r, os.path.join(OUTDIR, 'transcript.json'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.extract_audio(a, os.path.join(OUTDIR, 'myaudio'))